In [ ]:
import os
import platform
import pandas as pd
import tensorflow as tf
import datetime

from modules.dataset_utils import DatasetUtils as dataset
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


#########################################################################################################################################################
# PARAMETROS DEFINIDOS EN FUNCION DEL USUARIO
#########################################################################################################################################################

# Ruta hacia los CSV del dataset
train_file_relative = './data/700Train.csv'
validation_file_relative = './data/700Validation.csv'
test_file_relative = './data/700Test.csv'

# Ruta al directorio de imágenes con máscara aplicada
masked_dir_relative = '../local/output_masked/'

model_dir_relative = "./models"

# Ruta relativa hacia la librería ai4eutils
ai4eutils_relative = "./repos/ai4eutils"

# Ruta relativa hacia la librería CameraTraps
CameraTraps_relative = "./repos/CameraTraps"

#########################################################################################################################################################
# PARAMETROS PARA CNN
#########################################################################################################################################################

IMG_HEIGHT = IMG_WEIGHT = 700
BATCH_SIZE = 32
SEED = 42

#########################################################################################################################################################
#########################################################################################################################################################

home = os.path.expanduser("~")

ai4utils = os.path.abspath(ai4eutils_relative)
CameraTraps = os.path.abspath(CameraTraps_relative)
train_file = os.path.abspath(train_file_relative)
validation_file = os.path.abspath(validation_file_relative)
test_file = os.path.abspath(test_file_relative)
masked_dir = os.path.abspath(masked_dir_relative)
model_dir = os.path.abspath(model_dir_relative)

try:
    os.environ['PYTHONPATH']
except KeyError:
    os.environ['PYTHONPATH'] = ""
if platform.system() == 'Windows':
    os.environ['PYTHONPATH'] += (";" + ai4utils)
    os.environ['PYTHONPATH'] += (";" + CameraTraps)
else:
    os.environ['PYTHONPATH'] += (":" + ai4utils)
    os.environ['PYTHONPATH'] += (":" + CameraTraps)

print('=======================================================================================================================================')
print('PYTHONPATH: ' + os.environ['PYTHONPATH'])
print('')
print('ai4eutils PATH: ' + '\t' + ai4utils)
print('CameraTraps PATH: ' + '\t' + CameraTraps)
print('Train CSV PATH: ' + '\t'  + train_file)
print('Validation CSV PATH: ' + '\t' + validation_file)
print('Test CSV PATH: ' + '\t' + '\t' + test_file)
print('masked PATH: ' + '\t' + '\t' + masked_dir)
print('models PATH: ' + '\t' + '\t' + model_dir)
print('=======================================================================================================================================')

In [ ]:
file_path, labels = dataset.load_dataset(train_file, masked_dir)
TRAIN_SAMPLES=len(labels)
train_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
train_dataset = train_dataset.map(dataset.load_image).map(dataset.resize_image).batch(BATCH_SIZE)

file_path, labels = dataset.load_dataset(validation_file, masked_dir)
VALIDATION_SAMPLES=len(labels)
validation_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
validation_dataset = validation_dataset.map(dataset.load_image).map(dataset.resize_image).batch(BATCH_SIZE)

file_path, labels = dataset.load_dataset(test_file, masked_dir)
TEST_SAMPLES=len(labels)
test_dataset = tf.data.Dataset.from_tensor_slices((file_path, labels))
test_dataset = test_dataset.map(dataset.load_image).map(dataset.resize_image).batch(BATCH_SIZE)

In [ ]:
model = keras.Sequential([
    layers.Input((700, 700, 3)),
    layers.Conv2D(32, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(128, 3, padding='valid', activation= 'relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    #layers.Conv2D(128, 3, padding='valid', activation= 'relu'),
    #layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    loss = [keras.losses.BinaryCrossentropy(from_logits=False)],
    optimizer = keras.optimizers.Adam(),
    metrics =  ['accuracy']
)
model.summary()

In [ ]:
history = model.fit(
            train_dataset, 
            epochs=100,
            validation_data=validation_dataset,
            batch_size=BATCH_SIZE
)

In [ ]:
ax = pd.DataFrame(data=history.history).plot(figsize=(15, 7))
ax.grid()
_ = ax.set(title="Training loss and accuracy", xlabel="Epochs")
_ = ax.legend(["Training loss", "Trainig accuracy", "Validation loss", "Validation accuracy"])

In [ ]:
results = model.evaluate(test_dataset, verbose=1)

In [ ]:
date = datetime.utcnow().strftime('%Y-%m-%d_%H-%M')
if platform.system() == 'Windows':
    path = model_dir + '\\'+ date
else:
    path = model_dir + '/'+ date

model.save(path)